# **Coleta de Dados de Acidentes de Trânsito no Brasil (2007-2024)**

Este notebook descreve o processo de coleta de dados sobre acidentes de trânsito no Brasil, utilizando informações fornecidas pelo Departamento Nacional de Trânsito (DETRAN). O objetivo é automatizar a coleta de arquivos CSV e organizá-los para análise e processamento posterior.

# 1. Importação de Bibliotecas e Configuração Inicial

Primeiramente, vamos importar as bibliotecas necessárias e configurar o ambiente de logging, que nos ajudará a monitorar o progresso e possíveis erros durante a execução.

In [1]:
import pandas as pd
import shutil
from pathlib import Path
import logging


C:\Users\joseb\AppData\Local\Temp\ipykernel_28968\4288788791.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Configuração do Log
A configuração do log permite que você registre mensagens em diferentes níveis (informação, erro, etc.), o que facilita o rastreamento da execução do processo.


In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Configuração do Log
A configuração do log permite que você registre mensagens em diferentes níveis (informação, erro, etc.), o que facilita o rastreamento da execução do processo.

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


# 2. Definição das Constantes

Aqui, definimos algumas constantes que serão utilizadas ao longo do código. Elas ajudam a manter o código mais legível e organizado.

In [4]:
COLUMN_DESCRIPTION = "DESCRIPTION"
COLUMN_URL = "URL"
COLUMN_YEAR = "YEAR"
COLUMN_FILE_NAME = "FILE_NAME"
MESSAGE_ERROR_FILE_EXIST = '/files já existe. Não será feito novo download'
FILTER_TEXT_DOCUMENT_CSV_ACIDENTES = 'Documento CSV de Acidentes '
FILTER_TEXT_ALL_CASES = "Todas as causas e"
FILTER_GROUP_FOR_PEOPLE = "Agrupados por pessoa"
PATH_SAVE_FILE_YAML = "paths.save_files"
URL_REPLACE = "https://drive.usercontent.google.com/u/0/uc?id=ID_FILE&export=download"


## 3. Funções para Coleta e Processamento de Dados

### Função para Verificar se a Pasta de Arquivos Já Existe

Antes de iniciar o download dos arquivos, precisamos verificar se a pasta de arquivos já existe. Caso contrário, a pasta será criada.

In [5]:
def __verify_if_file_folder_exist(force_download):
    folder_files = root_path / "files"
    if force_download and folder_files.exists() and folder_files.is_dir():
        shutil.rmtree(folder_files)
        return False
    return folder_files.is_dir()


### Função para Obter os Dados do Detran
Esta função obtém os dados do DETRAN em formato HTML e converte-os para um DataFrame.

In [6]:
def __getDataFrame():
    url = config.get("database.detran")
    df = pd.read_html(url, extract_links="body")
    df = df[1]  # Pega a segunda tabela
    df = df.iloc[1:-1]  # Exclui a primeira e última linha
    df.columns = [COLUMN_DESCRIPTION, COLUMN_URL]
    return df


### Função para Limpar o DataFrame

Aqui, realizamos a limpeza do DataFrame, ajustando as colunas e removendo informações indesejadas.

In [7]:
def __clean_dataframe(df):
    df.loc[:, "URL"] = df["URL"].apply(lambda x: x[1].split("/")[-3])
    df.loc[:, "URL"] = df["URL"].apply(lambda x: URL_REPLACE.replace("ID_FILE", x))
    df.loc[:, COLUMN_DESCRIPTION] = df[COLUMN_DESCRIPTION].apply(lambda x: x[0])
    df.loc[:, COLUMN_DESCRIPTION] = df[COLUMN_DESCRIPTION].str.replace(FILTER_TEXT_DOCUMENT_CSV_ACIDENTES, '', regex=False)
    df[COLUMN_YEAR] = df[COLUMN_DESCRIPTION].str.extract(r'(\d{4})')
    return df


## Função para Remover Linhas com Dados Repetidos

Esta função é responsável por remover qualquer dado repetido ou não agrupado corretamente.

In [8]:
def __remove_rows_with_data_repeat(df):
    def filtrar_links(df_g):
        if df_g[COLUMN_DESCRIPTION].str.contains('Todas as causas e', na=False).any():
            return df_g[df_g[COLUMN_DESCRIPTION].str.contains('Todas as causas e', na=False)]
        else:
            return df_g
    return df.groupby(COLUMN_YEAR).apply(filtrar_links, include_groups=True).reset_index(drop=True)


### Função para Gerar Nomes de Arquivos
A função abaixo gera nomes para os arquivos com base nas descrições das colunas, facilitando a organização dos dados.



In [9]:
def __generate_column_name_file(df):
    def get_name_file(col):
        year = col[:4]
        if FILTER_TEXT_ALL_CASES in col:
            return f"{year}_geral"
        elif FILTER_GROUP_FOR_PEOPLE in col:
            return f"{year}_agg_pessoa"
        else:
            return f"{year}_agg_ocorrencia"
    df.loc[:, COLUMN_FILE_NAME] = df[COLUMN_DESCRIPTION].apply(get_name_file)


### Função para Baixar e Salvar os Arquivos
Esta função faz o download dos arquivos CSV e os salva na pasta especificada.

In [10]:
def __download_and_save_files(df):
    errors = []
    for index, row in df.iterrows():
        url = row[COLUMN_URL]
        file_name = row[COLUMN_FILE_NAME]
        year = row[COLUMN_YEAR]
        if "_agg_ocorrencia" not in file_name:
            continue
        file_name_save = f"datatran{year}"
        download_folder = Path(__file__).parent.parent.parent / config.get(PATH_SAVE_FILE_YAML)
        file_is_save = FileDownloader.download_and_save(url, file_name_save, ".csv", download_folder)
        if not file_is_save:
            errors.append(year)
    for y in errors:
        logger.error(f"Dataset para o ano {y} não realizado.")


### Função Principal para Coletar Dados

Por fim, temos a função principal que coordena todo o processo de coleta e processamento de dados.

In [11]:
def execute_data_collection(force_download=False):
    """ Método principal para executar o fluxo de coleta e processamento de dados. """
    if __verify_if_file_folder_exist(force_download):
        logging.warning(f"Folder já existe. Não será feito novo download.")
        return logging.info("Iniciando coleta de dados")
    df = __getDataFrame()
    __clean_dataframe(df)
    __generate_column_name_file(df)
    __download_and_save_files(df)
    logging.info("Finalizando coleta de dados")


### 4. Execução do Processo de Coleta de Dados

Agora, vamos executar o processo de coleta de dados. Isso irá iniciar o download e salvar os arquivos necessários.

In [ ]:
execute_data_collection()
